In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
from datetime import datetime
import re
import os, threading
import requests

In [ ]:
def save_full_screenshot(driver, regnumber=""):
    try:
        os.makedirs("screenshots", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"screenshots/{regnumber}_{timestamp}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")
        
def scrape(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    # ====================================================================================================================
    # get the username tab
    try:
        provided_u_name = "EG78TP9"
        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Username"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@placeholder="Password"]')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@value="Login"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click() 
    except Exception as e:
        print(f"No check tab found and the error is {e}") 

    time.sleep(10)
    # # number of cars
    # try:
    #     cars =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h3[@class="sc-gqjmRU gwltCt"]'))).text.strip()
    #     if cars:
    #         cars = int(cars)
    #         print(f"{cars} number of cars found")
    # except Exception as e:
    #     print(f"No cars found")
    # try:
    #     page_cars =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/h3[@class="sc-gqjmRU gwltCt vehicle-card-titles__main__title__text"]')))
    #     if page_cars:
    #         driver.execute_script("arguments[0].scrollIntoView();", page_cars[0])
    #         time.sleep(1)
    #         page_cars[0].click()
    # except Exception as e:
    #     print(f"No car link to click {e}")
        
    # result
    results = []
    for _ in range(50): # cars
        if not driver.window_handles:
            print("Browser window closed. Stopping script.")
            break
            
        details= {}
        # car name--> done
        try:
            car_name = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, './/h2[contains(@class,"card-title")]'))).text.strip()
            car_sub_name = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, './/h3[contains(@class,"card-subtitle")]'))).text.strip()
            if car_name or car_sub_name:
                details['Title'] = car_name
                details['Sub_Title'] = car_sub_name
            else:
                details['Title'] = "na"
                details['Sub_Title'] ="na"
            
        except Exception as e:
            print("No car name")
        
        
        
        # image card --> done   
        try:
            img_clk = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, './/div[contains(@class, "gallery__image")]'))
            )
            img_clk.click()
            time.sleep(1)  # Allow modal to load

            imgs = []
            last_src = None

            while True:
                try:
                    # Wait for image container
                    img_card = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, './/div[contains(@class, "modal-gallery__body")]'))
                    )
                    img = img_card.find_element(By.TAG_NAME, "img")
                    src = img.get_attribute("src")

                    # Only add if it's a new image
                    if src and src != last_src:
                        imgs.append(src)
                        last_src = src
                        print(f"📸 Captured image: {src}")

                    # Try clicking next
                    next_butn = driver.find_elements(By.XPATH, './/button[contains(@class, "byahoa")]')
                    if next_butn:
                        next_butn[0].click()
                        time.sleep(0.5)  # Give time for next image to load
                    else:
                        print("🚫 No more next button found.")
                        break

                except Exception as e:
                    print(f"⚠️ No more images or error: {e}")
                    break

            # Close modal if open
            try:
                close = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, './/button[text()="Close"]'))
                )
                close.click()
            except Exception:
                pass

            # Store image links as comma-separated string
            details["Images"] = ", ".join(imgs)
            print(f"✅ Found {len(imgs)} images.")

        except Exception as e:
            print(f"❌ Error processing image card: {e}")
        
            
        # bid pricing --> done
        try:
            pr_main_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-banner__purchase-block"]')))
            if pr_main_card:
                pr_lbl = pr_main_card.find_element(By.XPATH, './/label[@class="sc-gzVnrw kiynVB pricing-block__header"]').text.strip()
                pr_val = pr_main_card.find_element(By.XPATH, './/p[contains(@class, "sc-kAzzGY")]').text.strip()
                if pr_lbl and pr_val:
                    details[pr_lbl] = pr_val
        except Exception as e:
            print(f"No pricings")
                
        
        # getting tabs information
        try:
            tabs_clck_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-header-block"]')))
            if tabs_clck_card:
            
                # get the veh details --> done
                try:
                    veh_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Vehicle Details"]')))
                    if veh_det_clck: 
                        driver.execute_script("arguments[0].scrollIntoView();", veh_det_clck)
                        veh_det_clck.click()
                        
                        # veh det informatin --> done
                        try:
                            veh_info_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-details-top-block"]')))
                            if veh_info_card:
                                veh_rows = WebDriverWait(veh_info_card,5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle-details-detail-list__row"]')))
                                if veh_rows:
                                    for veh_row in veh_rows:
                                        veh_lbl = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                        veh_val = veh_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                        if veh_lbl and veh_val:
                                            details[veh_lbl] = veh_val
                                        else:
                                            print("No veh label and vale")
                                else:
                                    print("No veh rows")
                            else:
                                print("No veh det card")
                        except Exception as e:
                            print("no veh det found")
                            
                        # specifictaion --> done
                        try:
                            spec_card = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, './/span[text()="View Technical Specification"]')))
                            if spec_card:
                                time.sleep(1)
                                spec_card.click()
                                
                                spec_det_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="accordion__content"]'))) 
                                if spec_det_card:
                                    spec_rows = WebDriverWait(spec_det_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="technical-spec-detail-list__row"]')))
                                    if spec_rows:
                                        for spec_row in spec_rows:
                                            spec_lbl = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[0].text.strip()
                                            spec_val = spec_row.find_elements(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]')[1].text.strip()
                                            if spec_lbl and spec_val:
                                                details[spec_lbl] = spec_val
                                            else:
                                                print("No spec label and vale")
                                    else:
                                        print("No spec rows")
                                else:
                                    print("No spec card")
                            else:
                                print("No spec button")
                        except Exception as e:
                            print(" No spec card found")
                    else:
                        print("No veh det card to click")
                except Exception as e:
                    print("No veh card")
                
                    
                # Standard Equipment --> done
                try:
                    stnd_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Standard Equipment"]')))
                    if stnd_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", stnd_det_clck)
                        stnd_det_clck.click()

                        stnd_info_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if stnd_info_card:
                            stnd_rows = WebDriverWait(stnd_info_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="text-list__row"]')))
                            if stnd_rows:
                                for stnd_row in stnd_rows:
                                    stnd_lbl = stnd_row.find_element(By.XPATH, './/p[@class="sc-htpNat cNoVxz"]').text.strip()
                                    stnd_vals = stnd_row.find_elements(By.TAG_NAME, 'li')
                                    if stnd_vals:
                                        stnd_val_lst = [li.text.strip() for li in stnd_vals]
                                        if stnd_lbl and stnd_val_lst:
                                            details[stnd_lbl] = stnd_val_lst
                                        else:
                                            print("No standard labels and values")
                                            
                            else:
                                print("No standard rows")
                        else:
                            print("No standard card")
                    else:
                        print("No veh det card to click")
                except Exception as e:
                    print(f"No standard card")
                
                # Inspection Report --> done
                try:
                    inspec_det_clck = WebDriverWait(tabs_clck_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Inspection Report"]')))
                    if inspec_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", inspec_det_clck)
                        inspec_det_clck.click()

                        inspec_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if inspec_card:
                            inspec_link = WebDriverWait(inspec_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Download PDF"]')))
                            if inspec_link:
                                inspec = inspec_link.get_attribute("href")
                                details['Inspection_report'] = inspec
                            else:
                                details['Inspection_report'] = "na"
                        else:
                            print("No inspec card")
                    else:
                        print("No inspec card to click")
                except Exception as e:
                    print(f"No inspec card")
                    
                # Other Documents --> done
                try:
                    other_det_clck = WebDriverWait(tabs_clck_card,3).until(EC.presence_of_element_located((By.XPATH, './/p[text()="Other Documents"]')))
                    if other_det_clck:
                        driver.execute_script("arguments[0].scrollIntoView();", other_det_clck)
                        other_det_clck.click()

                        other_card =  WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="tabbed-view-panel-block"]')))
                        if other_card:
                            other_link = WebDriverWait(other_card,3).until(EC.presence_of_element_located((By.XPATH, './/a[@class="downloadBtn gaEvent-download"]')))
                            if other_link:
                                other = other_link.get_attribute("href")
                                details['Other Documents'] = other
                            else:
                                details['Other Documents'] = "na"
                        else:
                            print("No Other Documents card")
                    else:
                        print("No Other Documents to click")
                except Exception as e:
                    print(f"No Other Documents")
                
        except Exception as e:
            print("No tabs to click")
        
        results.append(details)
        try:
            # print(details['Registration'])
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)  

            save_full_screenshot(driver, regnumber=details['Registration'])
            next_car =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/span[text() ="Next"]')))
            if next_car:
                driver.execute_script("arguments[0].scrollIntoView();", next_car)
                time.sleep(1)
                next_car.click()
                time.sleep(5)
            else:
                print("No more cars and no next btn found")
        except Exception as e:
            print(f"No more cars and no next btn found {e}")
            break
    df = pd.DataFrame.from_dict(results)
    df.to_csv("1link.csv", index=False)
    driver.quit()
path = "https://disposalnetwork.1link.co.uk/uk/tb/app/vehicle/25230179"
scrape(path)

📸 Captured image: https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/36086439?key=QGKQ87VDWrEetkWHxO4epEYtOORKZ5Abver%2BU0ACSxWh5aPZ4WWljpOcqQbYxtEQbVOyFQTcu%2FevtDQ6XIXVSQ%3D%3D
📸 Captured image: https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/36086440?key=rkFe6bkU2Uqr%2FT8s2wHJh5Qm3O%2F%2BGkIENWceZso2oSkvImAKJhotum3pWV%2BwpKJJjPEw6r23%2BHHEfL09gsdTkQ%3D%3D
📸 Captured image: https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/36086441?key=kHHMq7oOFAUl2xHx4LElWAauK7ybFmPXbu1pRvWbP0n6LPP5aGH%2BErQNHK%2FtkVHJO%2FqhNToKSDRH4QvuhGNcwg%3D%3D
📸 Captured image: https://disposalnetwork.1link.co.uk/uk/micro/artifacts/vehicleimage/36086442?key=Ys%2Fp%2BQ75yr3WRQ4Coc90V%2B1Ujqoj1eN%2BwANOX021obHN%2Fc2Sn2fQOoOAZlIeVGwoFZaAdr3Hr8Be%2B7JG%2BpL%2FCA%3D%3D
🚫 No more next button found.
✅ Found 4 images.
📸 Screenshot saved: screenshots/MW71YTN_20251110_182021.png


In [ ]:
df =pd.read_csv("1link.csv")
df.columns

Index(['Title', 'Sub_Title', 'Images', 'STARTING BID', 'Registration', 'Grade',
       'Condition', 'Fuel', 'Transmission', 'Registered',
       ...
       'Interior Lights', 'Safety', 'Seats', 'Security', 'Vanity Mirrors',
       'Wheels - Alloy', 'Wheels - Spare', 'Inspection_report',
       'Other Documents', 'BUY NOW'],
      dtype='object', length=118)

### images

In [ ]:
from urllib.parse import urlparse, urljoin
from PIL import Image, ImageDraw, ImageFont



def add_watermark_to_image(image_path, text="Sourced from 1link"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)
        font_size = max(5, image.width // 40)
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        margin = int(font_size * 0.6)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        box_padding = int(font_size * 0.5)
        draw.rectangle(
            [x - box_padding, y - box_padding,
             x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 220)
        )
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 240))
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")
    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")



def download_images(data, main_folder="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = row["Registration"]

        if pd.isna(row["Images"]) or not str(row["Images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
        else:
            image_urls = row["Images"].split(", ")
            reg_folder = os.path.join(main_folder, reg_no)
            os.makedirs(reg_folder, exist_ok=True)

            for idx, url in enumerate(image_urls):
                url = url.strip()
                if not url.startswith(("http://", "https://")):
                    url = urljoin("https://", url)

                parsed_url = urlparse(url)
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}")
                    continue

                try:
                    response = requests.get(url, stream=True)
                    response.raise_for_status()

                    file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                    file_extension = file_name.split(".")[-1]
                    if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                        file_name = f"image_{idx + 1}.jpg"

                    full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                    with open(full_file_name, 'wb') as f:
                        for chunk in response.iter_content(1024):
                            f.write(chunk)

                    print(f"✅ Downloaded: {full_file_name}")

           
                    add_watermark_to_image(full_file_name)

                except Exception as e:
                    print(f"❌ Failed to download {url} for {reg_no}: {e}")



def download_reports(data, main_folder="Inspec_report"):
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = row["Registration"]
        save_full_screenshot(driver, "after_login")
        report_urls = row["Inspection_report"]

        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")
        else:
            if not report_urls.startswith(("http://", "https://")):
                report_urls = urljoin("https://", report_urls)

            parsed_url = urlparse(report_urls)
            try:
                response = requests.get(report_urls, stream=True)
                response.raise_for_status()

                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf"

                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                print(f"✅ Downloaded: {full_file_name}")
            except Exception as e:
                print(f"❌ Failed to download {report_urls} for {reg_no}: {e}")



def start_funcs():
    thread1 = threading.Thread(target=download_images, args=(reg_img,))
    thread2 = threading.Thread(target=download_reports, args=(reports,))

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()


if __name__ == "__main__":
    df = pd.read_csv("1link.csv")
    reg_img = df[['Registration', 'Images']]
    reports = df[['Registration', 'Inspection_report']]
    start_funcs()


Exception in thread Thread-4 (download_reports):
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_3708\3018317760.py", line 91, in download_reports
    save_full_screenshot(driver, "after_login")
                         ^^^^^^
NameError: name 'driver' is not defined


✅ Downloaded: Images\MW71YTN\MW71YTN_1.jpg
✅ Watermark added to Images\MW71YTN\MW71YTN_1.jpg
✅ Downloaded: Images\MW71YTN\MW71YTN_2.jpg
✅ Watermark added to Images\MW71YTN\MW71YTN_2.jpg
✅ Downloaded: Images\MW71YTN\MW71YTN_3.jpg
✅ Watermark added to Images\MW71YTN\MW71YTN_3.jpg
✅ Downloaded: Images\MW71YTN\MW71YTN_4.jpg
✅ Watermark added to Images\MW71YTN\MW71YTN_4.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_1.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_1.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_2.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_2.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_3.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_3.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_4.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_4.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_5.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_5.jpg
✅ Downloaded: Images\RJ72MYU\RJ72MYU_6.jpg
✅ Watermark added to Images\RJ72MYU\RJ72MYU_6.jpg
❌ Failed to download https://disposalnetwork.1link.co.uk/uk/micro/arti